<a href="https://colab.research.google.com/github/H12324/APS360-Project/blob/main/APS360_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from skimage import io, color
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageOps
import numpy as np
from skimage.transform import rescale, resize, downscale_local_mean
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import random
import scipy.signal as sg
import os


!pip install timm #for CSPDarknet52 pretrained
import timm

!pip install torchinfo #for generating model summaries
from torchinfo import summary

!pip install piqa #for psnr/ssim
import piqa

# Dataset Creation

In [ ]:
# OUTPUT SAVED AS NPARRAY!! (because all image objects in all libraries are RGB afaik)

# input: folder of RGB images
# output: folder of LAB image, with padding and cropped 

def preprocess_images(input_path, output_path): 
  if not os.path.exists(output_path): 
    os.mkdir(output_path)
  with os.scandir(input_path) as entries:
    count =0
    for entry in entries:
        path_name = input_path+ '/'+ entry.name
      #  print(path_name)
    #    rgb = io.imread(path_name)
        rgb = Image.open(path_name)
        rgbplot = plt.imshow(rgb)
        plt.show()

        desired_size = 640

        old_size = rgb.size  # old_size is in (width, height) format
        print(old_size)
        delta_w = desired_size - old_size[0]
        delta_h = desired_size - old_size[1]
        padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
        rgb = ImageOps.expand(rgb, padding) # padding done

        rgb_resized = rgb.resize((256,256)) #resizing
        resized = plt.imshow(rgb_resized)
        plt.show()
        
        rgb_resized=np.asarray(rgb_resized)
        rgb_resized = rgb_resized[:, :, :3]
    #    print(rgb_resized.shape)
        lab = color.rgb2lab(rgb_resized)

   #     print(lab.shape)
        rgbAgain = color.lab2rgb(lab)
        rgbAgainplot = plt.imshow(rgbAgain)
        plt.show()

        labcopy = lab
        labcopy[:, :, 1] = 0
        labcopy[:, :, 2] = 0
        grey = color.lab2rgb(labcopy)
        greyplot = plt.imshow(grey)
        plt.show()

      #  img = Image.fromarray(lab)
        np.save(output_path+'/'+entry.name, lab)


##For creating COCO based dataset

In [ ]:
!wget "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
!unzip "/content/annotations_trainval2017.zip"

In [ ]:
#from https://stackoverflow.com/questions/51100191/how-can-i-download-a-specific-part-of-coco-dataset
from pycocotools.coco import COCO
import requests
# instantiate COCO specifying the annotations json path
coco = COCO('/content/annotations/instances_train2017.json')

In [ ]:
%%shell
mkdir coco_person_raw

In [ ]:
#!rm coco_person_raw -rf 

In [ ]:
catIds = coco.getCatIds(catNms=['person']) # we focus on people

imgIds = coco.getImgIds(catIds=catIds)
print (imgIds)

In [ ]:
samples = 1600 # number to download

# Specify a list of category names of interest
catIds = coco.getCatIds(catNms=['person']) # we focus on people
# Get the corresponding image ids and images using loadImgs
imgIds = coco.getImgIds(catIds=catIds)
images = coco.loadImgs(imgIds)
print(images[:1]) #to show how it works
# Save the images into a local folder
i=1
for im in random.sample(images, samples):
    img_data = requests.get(im['coco_url']).content
    with open('/content/coco_person_raw/' + im['file_name'], 'wb') as handler:
       handler.write(img_data)
       print(i)
       i+=1

##Create dataset folders (train/val/test)
(also does resizing + padding if needed and not already preprocessed)

In [ ]:
#exports dataset in lab format with padding and resizing
from skimage import color

# preprocess_images("/content/landscapes_dataset/archive","/content/landscape_preprocessed")
random.seed(100)
folders = ['train/', 'test/','val/']

#source = '/content/coco_person_raw/'
#dest_folders = ['/content/small_dataset/' + i for i in folders] 

source = '/content/landscapes_dataset/archive'
dest_folders = ['/content/large_dataset/' + i for i in folders]

samples = 4320 #total images to be split, must be divisible by 32

files =  os.listdir(source)
train_imgs = files[:int(0.8*samples)]
val_imgs = files[int(0.8*samples):int(0.9*samples)]
test_imgs = files[int(0.9*samples):]
print (len(train_imgs), len(val_imgs), len(test_imgs))

small_size = 128 #resize to 128 x 128

print(test_imgs)
for i in dest_folders:
  os.makedirs(i, exist_ok= True)

for img in train_imgs: 
  rgb = Image.open(os.path.join(source,img))
  old_size = rgb.size  # old_size is in (width, height) format
  desired_size = max(old_size)
  delta_w = desired_size - old_size[0]
  delta_h = desired_size - old_size[1]
  padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
  new_im = ImageOps.expand(rgb, padding) # padding done
  new_im = new_im.resize((small_size,small_size)) # resizing done
  # https://stackoverflow.com/questions/3228361/using-pil-and-numpy-to-convert-an-image-to-lab-array-modify-the-values-and-then
  npArray = np.asarray(new_im)
  if npArray.shape[-1] == 3:
    labArray = color.rgb2lab(npArray) # Convert array from RGB into Lab
    np.save(os.path.join(dest_folders[0],img.split('.')[0]),labArray) # saving
  else:
    print (f"{img.split('.')[0]} not used")  
 


for img in val_imgs: 
  rgb = Image.open(os.path.join(source,img))
  old_size = rgb.size  # old_size is in (width, height) format
  delta_w = desired_size - old_size[0]
  delta_h = desired_size - old_size[1]
  padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
  new_im = ImageOps.expand(rgb, padding) # padding done
  new_im = new_im.resize((small_size,small_size)) # resizing done
  # https://stackoverflow.com/questions/3228361/using-pil-and-numpy-to-convert-an-image-to-lab-array-modify-the-values-and-then
  npArray = np.asarray(new_im)
  labArray = color.rgb2lab(npArray) # Convert array from RGB into Lab
  np.save(os.path.join(dest_folders[2],img.split('.')[0]),labArray) # saving
  
for img in test_imgs: 
  rgb = Image.open(os.path.join(source,img))
  old_size = rgb.size  # old_size is in (width, height) format
  delta_w = desired_size - old_size[0]
  delta_h = desired_size - old_size[1]
  padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
  new_im = ImageOps.expand(rgb, padding) # padding done
  new_im = new_im.resize((small_size,small_size)) # resizing done
  # https://stackoverflow.com/questions/3228361/using-pil-and-numpy-to-convert-an-image-to-lab-array-modify-the-values-and-then
  npArray = np.asarray(new_im)
  labArray = color.rgb2lab(npArray) # Convert array from RGB into Lab
  np.save(os.path.join(dest_folders[1],img.split('.')[0]),labArray) # saving
print("done")

##Dataset Helper Functions

In [ ]:
#Helper function to save single image with 

rgb = Image.open("/content/landscapes_dataset/archive/00000000.jpg")
old_size = rgb.size  # old_size is in (width, height) format
delta_w = desired_size - old_size[0]
delta_h = desired_size - old_size[1]
padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
new_im = ImageOps.expand(rgb, padding) # padding done
new_im = new_im.resize((small_size,small_size)) # resizing done
# https://stackoverflow.com/questions/3228361/using-pil-and-numpy-to-convert-an-image-to-lab-array-modify-the-values-and-then
npArray = np.asarray(new_im)
labArray = color.rgb2lab(npArray) # Convert array from RGB into Lab
np.save(os.path.join(dest_folders[0],img.split('.')[0]),labArray) # saving

In [ ]:
# can be used to convert lab to rgb effectively for single image
img_array = np.load('/content/small_dataset/train/000000000673.npy')
end = color.lab2rgb(img_array)*255
end = end.astype(np.uint8)
end = Image.fromarray(end, "RGB")
end = plt.imshow(end)

print(img_array.shape)

In [ ]:
# Save a copy of the small dataset so you dont have to repeat all of the above everytime
!zip -r large_dataset.zip large_dataset
from google.colab import files
files.download("/content/large_dataset.zip")

In [ ]:
# DELETE DATASET
!rm -r /content/small_dataset/

In [ ]:
############ Testing ####################
train_loader, val_loader = split_data('/content/large_dataset', 16)

print(train_loader)

for inputs, labels in val_loader:
  print('\n')
  print(inputs.shape)
  print(labels.shape)
  print('\n')

  to_rgb(inputs[0], labels[0])
  break

In [ ]:
### Needed omly when above fails ##########

path = "/content/landscapes_dataset/archive"
# Remove any partially unzipped file
input_filenames = os.listdir(path)
for filename in input_filenames :
  if filename == ".ipynb_checkpoints":
    shutil.rmtree(os.path.join(path, filename))
    continue
  image = np.load(path + "/" + filename)
  if image.shape[2] != 3:
    print("Deleted image with shape: " + str(image.shape))
    os.remove(os.path.join(path, filename)) 

##Loading existing dataset


In [ ]:
#if testing with a set dataset
!unzip large_dataset.zip 

In [ ]:
print(len(os.listdir("/content/large_dataset/train")))

3440


In [ ]:
#IF FOLDER HAS .IPYNB CHECKPOINTS, TRAINING WILL STOP
!rm -r /content/large_dataset/train/.ipynb_checkpoints

#Helper fns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# For quickly visualising a few images from training and validation set

def quick_qualitative(model, dataset, samples):
  t_loader, v_loader = split_data(dataset, 1)

  numb_t = 0

  for inputs, labels in t_loader:
    numb_t += 1
    print("Demonstrating Some Train Set Results")
    inputs = inputs.cuda()
    labels = labels.cuda()
    embed = cspdarknet53(inputs)[-1].to(device)

    logits = model(inputs,embed)
    print("Attempt at Reconstruction")
    to_rgb(inputs.cpu()[0], logits.cpu().detach().numpy()[0])
    print("Original")
    to_rgb(inputs.cpu()[0], labels.cpu()[0])
    if (numb_t == samples):
      break
  numb_v = 0
  for inputs, labels in v_loader:
    numb_v += 1
    print("Demonstrating Val Set Results")
    inputs = inputs.cuda()
    labels = labels.cuda()
    embed = cspdarknet53(inputs)[-1].to(device)

    logits = model(inputs,embed)
    print("Attempt at Reconstruction")
    to_rgb(inputs.cpu()[0], logits.cpu().detach().numpy()[0])
    print("Original")
    to_rgb(inputs.cpu()[0], labels.cpu()[0])
    if (numb_v == samples):
      break
  

In [ ]:
# Grey LAB to 3 channel RGB function, NOT USED
def greyLAB2RGB(grayscale,batch_size): 
  #resultrgb = np.empty((batch_size, 3, 128, 128))
  resultrgb = []
  for i, bw in enumerate(grayscale):
    bw_reshaped = bw.reshape(128,128, 1)
    image_bwlab =  np.append(bw_reshaped, np.zeros((128, 128, 2)), axis = 2)
    bwrgb = color.lab2rgb(image_bwlab)*255
    # colour = color.lab2rgb(image_bwlab)*255
    # end = colour.astype(np.uint8)
    # end = Image.fromarray(end, "RGB")
    # plt.imshow(end)
    # plt.show()
    bwrgb_reshaped = bwrgb.reshape(3, 128, 128)
    # print(bwrgb_reshaped)
    resultrgb.append(list(bwrgb_reshaped))
    #np.append(resultrgb[i],bwrgb_reshaped)
  # return torch.from_numpy(resultrgb).float()
  # tensorver = torch.FloatTensor(resultrgb)
  # print(tensorver.size())
  return torch.FloatTensor(resultrgb)
# train_loader, val_loader = split_data('/content/small_dataset', 16)
# for inputs, labels in val_loader:
#   bw_rgb = greyLAB2RGB(inputs,16)




In [ ]:
## Trainer Helper - for saving data
def get_model_name(batch_size, learning_rate, epoch):
    path = "bs{}_lr{}_epoch{}".format(batch_size, learning_rate, epoch)
    return path

class AverageMeter(object):
  '''A handy class from the PyTorch ImageNet tutorial''' 
  def __init__(self):
    self.reset()
  def reset(self):
    self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
  img_array = np.zeros((128, 128, 3))
  img_array[:,:,0] = np.squeeze( np.swapaxes(grayscale_input, 0, 1) )
  img_array[:,:,1:] = np.swapaxes( np.swapaxes(ab_input, 0, 2), 0, 1 )

  colour = color.lab2rgb(img_array)*255
  end = colour.astype(np.uint8)
  end = Image.fromarray(end, "RGB")
  plt.imshow(end)
  plt.show()

  # DEBUGGING: Showing BW Image
  # presentable_bw = np.repeat( np.expand_dims(color.rgb2gray(colour/255.0), axis=2), 3, axis=2 )
  # plt.imshow(presentable_bw)
  # plt.show()

  # TODO: Fix Saving the file
  #if save_path is not None and save_name is not None: 
    #presentable_bw = np.repeat( np.expand_dims(color.rgb2gray(colour/255.0), axis=2), 3, axis=2 )
    #plt.imsave(arr=presentable_bw, fname='{}{}'.format(save_path['grayscale'], save_name))
    #plt.imsave(arr=end, fname='{}{}'.format(save_path['colorized'], save_name))

#Training

##models which were used for inspiration/in development



In [ ]:
#UNET implementatino for inspiration, from https://towardsdatascience.com/u-net-b229b32b4a71
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

class UNet(nn.Module):
    def contracting_block(self, in_channels, out_channels, kernel_size=3):
        block = torch.nn.Sequential(
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=out_channels),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(out_channels),
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=out_channels, out_channels=out_channels),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(out_channels),
                )
        return block
    
    def expansive_block(self, in_channels, mid_channel, out_channels, kernel_size=3):
            block = torch.nn.Sequential(
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=mid_channel),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(mid_channel),
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=mid_channel),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(mid_channel),
                    torch.nn.ConvTranspose2d(in_channels=mid_channel, out_channels=out_channels, kernel_size=3, stride=2, padding=1, output_padding=1)
                    )
            return  block
    
    def final_block(self, in_channels, mid_channel, out_channels, kernel_size=3):
            block = torch.nn.Sequential(
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=mid_channel),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(mid_channel),
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=mid_channel),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(mid_channel),
                    torch.nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channel, out_channels=out_channels, padding=1),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm2d(out_channels),
                    )
            return  block
    
    def __init__(self, in_channel, out_channel):
        super(UNet, self).__init__()
        #Encode
        self.conv_encode1 = self.contracting_block(in_channels=in_channel, out_channels=64)
        self.conv_maxpool1 = torch.nn.MaxPool2d(kernel_size=2)
        self.conv_encode2 = self.contracting_block(64, 128)
        self.conv_maxpool2 = torch.nn.MaxPool2d(kernel_size=2)
        self.conv_encode3 = self.contracting_block(128, 256)
        self.conv_maxpool3 = torch.nn.MaxPool2d(kernel_size=2)
        # Bottleneck
        self.bottleneck = torch.nn.Sequential(
                            torch.nn.Conv2d(kernel_size=3, in_channels=256, out_channels=512),
                            torch.nn.ReLU(),
                            torch.nn.BatchNorm2d(512),
                            torch.nn.Conv2d(kernel_size=3, in_channels=512, out_channels=512),
                            torch.nn.ReLU(),
                            torch.nn.BatchNorm2d(512),
                            torch.nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
                            )
        # Decode
        self.conv_decode3 = self.expansive_block(256, 256, 128)
        self.conv_decode2 = self.expansive_block(128, 128, 64)
        self.final_layer = self.final_block(64, 64, out_channel)
        
    def crop_and_concat(self, upsampled, bypass, crop=False):
        if crop:
            c = (bypass.size()[2] - upsampled.size()[2]) // 2
            bypass = F.pad(bypass, (-c, -c, -c, -c))
        return torch.cat((upsampled, bypass), 1)
    
    def forward(self, x):
        # Encode
        encode_block1 = self.conv_encode1(x)
        encode_pool1 = self.conv_maxpool1(encode_block1)
        encode_block2 = self.conv_encode2(encode_pool1)
        encode_pool2 = self.conv_maxpool2(encode_block2)
        encode_block3 = self.conv_encode3(encode_pool2)
        encode_pool3 = self.conv_maxpool3(encode_block3)
        # Bottleneck
        bottleneck1 = self.bottleneck(encode_pool3)
        # Decode
        decode_block3 = self.crop_and_concat(bottleneck1, encode_block3)
        cat_layer2 = self.conv_decode3(decode_block3)
        decode_block2 = self.crop_and_concat(cat_layer2, encode_block2)
        cat_layer1 = self.conv_decode2(decode_block2)
        decode_block1 = self.crop_and_concat(cat_layer1, encode_block1)
        final_layer = self.final_layer(decode_block1)
        return  final_layer

In [ ]:
#our model with instance without unet res connections
class colorizer(nn.Module):
  def __init__(self):
    self.name = "colorizer"
    super(colorizer, self).__init__()
    self.Conv0 = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding = 'same'), # 128 * 128 * 32
    nn.ReLU(),
    nn.BatchNorm2d(32)
    )
    self.Conv1 = nn.Sequential(
    nn.Conv2d(32, 64, 3, stride = 2, padding = 1), # 64*64*64
    nn.ReLU(),
    nn.BatchNorm2d(64)
    )
    self.Conv2 = nn.Sequential(
    nn.Conv2d(64, 128, 2, padding = 'same'), # 64 * 64 * 128
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )
    self.Conv3 = nn.Sequential(
    nn.Conv2d(128, 256, 2, stride = 2, padding = 0), # 32 * 32 * 256
    nn.ReLU(),
    nn.BatchNorm2d(256)
    )
    self.Conv4 = nn.Sequential(
    nn.Conv2d(256, 512, 2,stride = 2, padding = 0), # 32 * 32 * 512
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.fused = nn.Sequential(
    nn.Conv2d(1024, 512, 2, padding = 'same'), # 32 * 32 * 512
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.Conv5 = nn.Sequential(
    nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256)
    )
    self.Conv6 = nn.Sequential(
    nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )
    """self.Conv6 = nn.Sequential(
    nn.ConvTranspose2d(256, 128, 2, output_padding = 32), # 64 * 64 * 128
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )"""
    self.Conv7 = nn.Sequential(
    nn.Conv2d(128, 64, 2, padding = 'same'), # 64 * 64 * 64
    nn.ReLU(),
    nn.BatchNorm2d(64)
    )
    self.Conv8 = nn.Sequential(
    nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(32)
    )
    """self.Conv8 = nn.Sequential(
    nn.ConvTranspose2d(64, 32, 2, output_padding = 64), # 128 * 128 * 32
    nn.ReLU(),
    nn.BatchNorm2d(64)"""
    
    self.Conv9 = nn.Sequential(
    nn.Conv2d(32, 2, 2, padding = 'same') # 128 * 128 * 2
    )

    
    self.instance_upscaled = nn.ConvTranspose2d(1024, 512, kernel_size=5, stride=4, padding=1, output_padding=1) # from 8x8 to 32x32


  def forward(self, x, embed):
    x = self.Conv0(x)
    x = self.Conv1(x)
    x = self.Conv2(x)
    x = self.Conv3(x)
    x = self.Conv4(x)
    embed = self.instance_upscaled(embed)
    #print (x.size(), embed.size())
    fuse = torch.cat((x, embed),1)
    x = self.fused(fuse)
    x = self.Conv5(x)
    x = self.Conv6(x)
    x = self.Conv7(x)
    x = self.Conv8(x)
    x = self.Conv9(x)
    return x


In [ ]:
#our model with instance without unet res connections
class colorizer_transpose(nn.Module):
  def __init__(self):
    self.name = "colorizer_transpose"
    super(colorizer_transpose, self).__init__()
    self.Conv0 = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding = 'same'), # 128 * 128 * 32
    nn.ReLU(),
    nn.BatchNorm2d(32)
    )
    self.Conv1 = nn.Sequential(
    nn.Conv2d(32, 64, 3, stride = 2, padding = 1), # 64*64*64
    nn.ReLU(),
    nn.BatchNorm2d(64)
    )
    self.Conv2 = nn.Sequential(
    nn.Conv2d(64, 128, 2, padding = 'same'), # 64 * 64 * 128
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )
    self.Conv3 = nn.Sequential(
    nn.Conv2d(128, 256, 2, stride = 2, padding = 0), # 32 * 32 * 256
    nn.ReLU(),
    nn.BatchNorm2d(256)
    )
    self.Conv4 = nn.Sequential(
    nn.Conv2d(256, 512, 2,stride = 2, padding = 0), # 32 * 32 * 512
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.fused = nn.Sequential(
    nn.Conv2d(1024, 512, 2, padding = 'same'), # 32 * 32 * 512
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.Conv5 = nn.Sequential(
    nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256)
    )
    self.Conv6 = nn.Sequential(
    nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )
    """self.Conv6 = nn.Sequential(
    nn.ConvTranspose2d(256, 128, 2, output_padding = 32), # 64 * 64 * 128
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )"""
    self.Conv7 = nn.Sequential(
    nn.Conv2d(128, 64, 2, padding = 'same'), # 64 * 64 * 64
    nn.ReLU(),
    nn.BatchNorm2d(64)
    )
    self.Conv8 = nn.Sequential(
    nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(32)
    )
    """self.Conv8 = nn.Sequential(
    nn.ConvTranspose2d(64, 32, 2, output_padding = 64), # 128 * 128 * 32
    nn.ReLU(),
    nn.BatchNorm2d(64)"""
    
    self.Conv9 = nn.Sequential(
    nn.Conv2d(32, 2, 2, padding = 'same') # 128 * 128 * 2
    )

    
    self.instance_upscaled = nn.ConvTranspose2d(1024, 512, kernel_size=5, stride=4, padding=1, output_padding=1) # from 8x8 to 32x32


  def forward(self, x, embed):
    x = self.Conv0(x)
    x = self.Conv1(x)
    x = self.Conv2(x)
    x = self.Conv3(x)
    x = self.Conv4(x)
    #embed = self.instance_upscaled(embed)
    #print (x.size(), embed.size())
    #fuse = torch.cat((x, embed),1)
    #x = self.fused(fuse)
    x = self.Conv5(x)
    x = self.Conv6(x)
    x = self.Conv7(x)
    x = self.Conv8(x)
    x = self.Conv9(x)
    return x


In [ ]:
# testing the embedding upsampling layer
class upscaler(nn.Module):
  def __init__(self):
    self.name = "upscaler"
    super(upscaler, self).__init__()
    self.instance_upscaled = nn.ConvTranspose2d(1024, 512, kernel_size=5, stride=4, padding=1, output_padding=1) # from 8x8 to 32x32

  def forward(self,x):
    x=self.instance_upscaled(x)
    return x


In [ ]:
model = upscaler()
model.to('cuda')

summary(model, input_size=(1, 1024, 8, 8))

Layer (type:depth-idx)                   Output Shape              Param #
upscaler                                 --                        --
├─ConvTranspose2d: 1-1                   [1, 512, 32, 32]          13,107,712
Total params: 13,107,712
Trainable params: 13,107,712
Non-trainable params: 0
Total mult-adds (G): 13.42
Input size (MB): 0.26
Forward/backward pass size (MB): 4.19
Params size (MB): 52.43
Estimated Total Size (MB): 56.89

In [ ]:
from timm.models.layers.activations import tanh
# OUR MODEL
class colorizer_UNET(nn.Module):           
  def __init__(self):
    self.name = "colorizer_UNET"
    super(colorizer_UNET, self).__init__()
    self.instance_upscale = nn.Sequential(
    nn.ConvTranspose2d(1024, 512, kernel_size=5, stride=4, padding=1, output_padding=1), # from 8x8 to 32x32
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.Conv0 = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding = 'same'), # 128 * 128 * 32 output
    nn.ReLU(),
    nn.BatchNorm2d(32)
    )
    self.Conv1 = nn.Sequential(
    nn.Conv2d(32, 64, 3, stride = 2, padding = 1), # 64 * 64 * 64 output
    nn.ReLU(),
    nn.BatchNorm2d(64)
    )
    self.Conv2 = nn.Sequential(
    nn.Conv2d(64, 128, 2, padding = 'same'), # 64 * 64 * 128 output
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )
    self.Conv3 = nn.Sequential(
    nn.Conv2d(128, 256, 2, stride = 2, padding = 0), # 32 * 32 * 256 output
    nn.ReLU(),
    nn.BatchNorm2d(256)
    )
    self.Conv4 = nn.Sequential(
    nn.Conv2d(256, 512, 2, padding = 'same'), # 32 * 32 * 512 output
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.Conv5 = nn.Sequential(
    nn.Conv2d(512, 512, 2, stride = 2, padding = 0), # 16 * 16 * 512 output
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.fusion = nn.Sequential(
    nn.Conv2d(1024, 512, 2, padding = 'same'), # 16 * 16 * 512+512 (embed + conv5) input
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.postfusion =nn.Sequential(
    nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, output_padding=1), # 32 * 32 output, 256+256 input
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.Conv6 = nn.Sequential(
    nn.Conv2d(1024, 256, 2, padding = 'same'), # 16 * 16 * 512+512 (fusion + conv4) input
    nn.ReLU(),
    nn.BatchNorm2d(256)
    )
    # self.Conv7 = nn.Sequential(
    # nn.Upsample(scale_factor=2),
    # nn.Conv2d(512, 128, 2, padding = 'same'), # 64 * 64 * 128
    # nn.ReLU(),
    # nn.BatchNorm2d(128)
    # )
    self.Conv7 = nn.Sequential(
    nn.ConvTranspose2d(512, 128, kernel_size=3, stride=2, padding=1, output_padding=1), # 32 * 32 output, 256+256 input
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )
    self.Conv8 = nn.Sequential(
    nn.Conv2d(256, 64, 2, padding = "same"), # 64 * 64 * 128 
    nn.ReLU(),
    nn.BatchNorm2d(64)
    )
    self.Conv9= nn.Sequential(
    nn.ConvTranspose2d(128, 32, kernel_size=3, stride=2, padding=1, output_padding=1), # 128 * 128 * 128
    nn.ReLU(),
    nn.BatchNorm2d(32)
    ) 
    self.Conv10 = nn.Sequential(
    nn.Conv2d(64, 2, 2, padding = 'same')#, # 128 * 128 * 2
    #nn.Tanh()
    )

    

  def forward(self, x, instance_embed):
    encode1 = self.Conv0(x)
    encode2 = self.Conv1(encode1)
    encode3 = self.Conv2(encode2)
    encode4 = self.Conv3(encode3)
    encode5 = self.Conv4(encode4)
    bottleneck = self.Conv5(encode5)
    instance_embed = self.instance_upscale(instance_embed)
    fused = torch.cat((instance_embed,bottleneck),1)
    fusion = self.fusion(fused) 
    fusion = self.postfusion(fusion)
    fused = torch.cat((fusion, encode5), 1) 
    decode1 = self.Conv6(fused)
    decode1 = torch.cat((decode1, encode4), 1)
    decode2 = self.Conv7(decode1)
    decode2 = torch.cat((decode2, encode3), 1)
    decode3 = self.Conv8(decode2)
    #print(decode3.size(),encode2.size())

    decode3 = torch.cat((decode3, encode2), 1)
    decode4 = self.Conv9(decode3)
    decode4 = torch.cat((decode4, encode1), 1)
    decode5 = self.Conv10(decode4)
    return decode5


##Training Helper functions

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
############# Alternate Loss Functions ##################
''' SSIM:
Source: https://stackoverflow.com/questions/53956932/use-pytorch-ssim-loss-function-in-my-model
'''
##### SSIM LOSS ############
class SSIMLoss(piqa.ssim.SSIM):
    def forward(self, x, y):
        return 1. - super().forward(x, y)

class PSNRLoss(piqa.psnr.PSNR):
    def forward(self, x, y):
        return piqa.psnr.psnr(x, y, value_range=128).mean()

In [ ]:
def get_accuracy(model, val_loader, embed, criterion, model_path, epoch, save_images):   
  model.eval()
  losses = AverageMeter()
  with torch.no_grad():
    for i, (input_gray, input_ab) in enumerate(val_loader):
      input_gray, input_ab = input_gray.cuda(), input_ab.cuda()

      output_ab = model(input_gray,embed)
      loss = criterion (output_ab,input_ab)
      losses.update(loss.item(), input_gray.size(0))
      if save_images:
        for j in range(min(len(output_ab), 5)): # save at most 5 images
            save_path = {'grayscale': str(model_path) + 'outputs/gray/', 'colorized': str(model_path)+'outputs/color/'}
            save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
            to_rgb(input_gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)
  return losses.avg

In [ ]:
def split_data(data_path, batch_size=1) :
  # Training
  train_transforms = transforms.Compose([])#transforms.Compose([transforms.Resize(128)])
  train_imagefolder = MockData(data_path + '/train', train_transforms)
  train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=False)

  # Validation 
  val_transforms = transforms.Compose([])
  val_imagefolder = MockData(data_path + '/val' , val_transforms)
  val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)
  return train_loader, val_loader

In [ ]:
## MOCK Dataloader ##
## Resources: 
# https://towardsdatascience.com/dataloader-for-sequential-data-using-pytorch-deep-learning-framework-part-2-ed3ad5f6ad82
# https://medium.com/analytics-vidhya/writing-a-custom-dataloader-for-a-simple-neural-network-in-pytorch-a310bea680af
# https://blog.paperspace.com/dataloaders-abstractions-pytorch/
class MockData(torch.utils.data.Dataset):   
   def __init__(self, path, transforms):
      super(torch.utils.data.Dataset, self).__init__()
      self.input_files = os.listdir(path) # e.g. content/small_dataset/train
      self.transform = transforms
      self.path = path
   def __len__(self):
        return len(self.input_files)
   def __getitem__(self, index):
       input_filename = self.input_files[index]
       ''' Code before we saved lab images
       print(input_filename)
       image = Image.open(os.path.join(self.path, input_filename))

       image = transforms.ToTensor()(image)
       #print(self.input_files[index], image.shape)

       image =  transforms.Resize((128, 128))(image)
       image = torch.swapaxes( torch.swapaxes(image, 0, 2), 0, 1)
       image = color.rgb2lab(1.0/255*image)
       image = torch.swapaxes(torch.from_numpy(image), 0, 2)

       image_bw = image[0,:,:]
       image_bw = torch.unsqueeze( image_bw, 0)
       label = image[1:,:,:]
       label /= 128

       #print(image_bw)
       #print(label)
       '''

       image = np.load(os.path.join(self.path, input_filename))

       image_bw = image[:,:,0]
       label = image[:,:,1:]

       image_bw = image_bw.reshape(1, 128, 128)
       label = np.swapaxes( np.swapaxes(label, 0, 2 ), 2, 1)

       image_bw = torch.from_numpy(image_bw).float()
       label = torch.from_numpy(label).float()

       return image_bw, label 

##Model and training

In [ ]:
cspdarknet53 = timm.create_model('cspdarknet53', pretrained=True, features_only = True, in_chans=1)
#cspdarknet53.to('cuda')

summary(cspdarknet53, input_size=(1,1,128,128))

In [ ]:
# OUR MODEL v2
class colorizer_UNETv2(nn.Module):           
  def __init__(self):
    self.name = "colorizer_UNETv2"
    super(colorizer_UNETv2, self).__init__()
    self.instance_upscale = nn.Sequential(
    nn.ConvTranspose2d(1024, 512, kernel_size=5, stride=4, padding=1, output_padding=1), # from 8x8 to 32x32
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.Conv0 = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding = 'same'), # 128 * 128 * 32 output
    nn.ReLU(),
    nn.BatchNorm2d(32)
    )
    self.Conv1 = nn.Sequential(
    nn.Conv2d(32, 64, 3, stride = 2, padding = 1), # 64 * 64 * 64 output
    nn.ReLU(),
    nn.BatchNorm2d(64)
    )
    self.Conv2 = nn.Sequential(
    nn.Conv2d(64, 128, 2, padding = 'same'), # 64 * 64 * 128 output
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )
    self.Conv3 = nn.Sequential(
    nn.Conv2d(128, 256, 2, stride = 2, padding = 0), # 32 * 32 * 256 output
    nn.ReLU(),
    nn.BatchNorm2d(256)
    )
    self.Conv4 = nn.Sequential(
    nn.Conv2d(256, 512, 2, padding = 'same'), # 32 * 32 * 512 output
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.bottleneck1 = nn.Sequential(
    nn.Conv2d(512, 512, 2, stride = 2, padding = 0), # 16 * 16 * 512 output
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.fusion = nn.Sequential(
    nn.Conv2d(1024, 512, 2, padding = 'same'), # 16 * 16 * 512+512 (embed + conv5) input
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.bottleneck2 = nn.Sequential(
    nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, output_padding=1), # 32 * 32 output, 512+512 input
    nn.ReLU(),
    nn.BatchNorm2d(512)
    )
    self.decode4 =nn.Sequential(
    nn.Conv2d(1024, 256, 2, padding = 'same'), # 32 * 32 * 512 input
    nn.ReLU(),
    nn.BatchNorm2d(256)
    )
    self.decode3 = nn.Sequential(
    nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1), # 32 * 32 output, 256+256 input
    nn.ReLU(),
    nn.BatchNorm2d(128)
    )
    self.decode2 = nn.Sequential(
    nn.Conv2d(256, 64, 2, padding = "same"), # 64 * 64 * 128 
    nn.ReLU(),
    nn.BatchNorm2d(64)
    )
    self.decode1= nn.Sequential(
    nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1), # 128 * 128 * 128
    nn.ReLU(),
    nn.BatchNorm2d(32)
    ) 
    self.decode0 = nn.Sequential(
    nn.Conv2d(64, 2, 2, padding = 'same')#, # 128 * 128 * 2
    #nn.Tanh()
    )

    

  def forward(self, x, instance_embed):
    encode1 = self.Conv0(x)
    encode2 = self.Conv1(encode1)
    encode3 = self.Conv2(encode2)
    encode4 = self.Conv3(encode3)
    encode5 = self.Conv4(encode4)
    bottleneck = self.bottleneck1(encode5)
    instance_embed = self.instance_upscale(instance_embed)
    fused1 = torch.cat((instance_embed,bottleneck),1)
    fusion = self.fusion(fused1) 
    bottleneck2 = self.bottleneck2(fusion)
    fused2 = torch.cat((bottleneck2, encode5), 1) 
    decode4 = self.decode4(fused2)
    decode3 = self.decode3(decode4)
    fused3 = torch.cat((decode3, encode3), 1)
    decode2 = self.decode2(fused3)
    decode1 = self.decode1(decode2)
    fused4 = torch.cat((decode1, encode1), 1)
    decode0 = self.decode0(fused4)
    return decode0


In [ ]:
def train(model, dataset_path, learning_rate=0.01, batch_size=64, num_epochs=1):
    model = model.to(device)
    train_loader, val_loader = split_data(dataset_path, batch_size)
    criterion = PSNRLoss().to(device) # nn.MSELoss().to(device) # Alternatives, PSNR, PA (https://core.ac.uk/download/pdf/151072499.pdf) 
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5, verbose= True, min_lr=1e-6)

    iters, train_losses, val_losses = [], [], []
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.7, verbose= True, min_lr=1e-6)
    model_name = get_model_name(batch_size, learning_rate, num_epochs) 
    model_path = os.path.join("/content/drive/MyDrive/APS360_final_training/backup", model_name)
    os.makedirs(model_path, exist_ok=True)

    start_time = time.time() 

    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
      model.train()
      for i, (bw_images,labels) in enumerate(train_loader):
        #bw_rgb = greyLAB2RGB(bw_images, batch_size).to(device)
        bw_images = bw_images.cuda()
        # print(bw_rgb.size())
        #bw_images = bw_images.cuda()
        labels = labels.cuda()
        
        embed = cspdarknet53(bw_images)[-1].to(device)
        
        #print(bw_images.shape, embed.size())
        logits = model(bw_images,embed)

        #to_rgb(bw_images.cpu()[0], labels.cpu()[0])
        #print("logits shape = " + str(logits.shape)) >> [batch_size, 2, 128, 128]

        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      # Display a few images before and after for each epoch
      if epoch % 2 == 1:
        #print("hello")
        iters.append(epoch)
        train_losses.append(float(loss))  
        val_losses.append(float(get_accuracy(model, val_loader, embed, criterion, model_path, epoch, False)))  # compute validation accuracy
        print("Epoch [{}/{}], Loss: {:.4f}, Training losses: {:.4f}, Validation losses: {:.4f}".format(epoch+1,num_epochs,loss,train_losses[-1],val_losses[-1]))
        scheduler.step(val_losses[-1])
        try: 
          if val_losses[-1]==min(val_losses): #model saved if performance is better 
            torch.save(model.state_dict(), model_path + "/" + "best")
            print("best model saved at epoch {}".format(epoch+1))
        except:
          continue
      if epoch%10 == 0: #model checkpoint every 20 epochs
        torch.save(model.state_dict(), model_path + "/" + get_model_name(batch_size, learning_rate, epoch))
        print ("checkpoint saved at epoch {}".format(epoch+1)) 
        quick_qualitative(model, "/content/large_dataset", 10)
    
    print("Total time taken: {}".format(time.time() - start_time))   
  
    # plotting
    plt.title("Training Curve")
    plt.plot(iters, train_losses, label="Train")
    plt.plot(iters, val_losses, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()
    plt.savefig(str(model_path) + '/training curve.png')


In [ ]:
#TRAINING
low_lr = colorizer_UNETv2()
low_lr.load_state_dict(torch.load("/content/land_miracle"))
train(low_lr, '/content/large_dataset', learning_rate=0.0005, batch_size=16, num_epochs=51)
# torch.save(model.state_dict(), "/content/drive/MyDrive/APS360_final_training/backup/bs16_lr0.01_epoch51/bs16_lr0.01_final")

#Testing


In [ ]:
# Train/Val Qualitative Results
train_loader, val_loader = split_data('/content/large_dataset', 1)
criterion = PSNRLoss().to(device)
total_loss = []

for inputs, labels in enumerate(val_loader): #change the loader here for train
  bw_rgb = greyLAB2RGB(inputs,1).cuda()
  inputs = inputs.cuda()
  print(inputs.size())
  labels = labels.cuda()
  embed = cspdarknet53(bw_rgb)[-1].to(device)

  logits = model(inputs,embed)
  loss = criterion(logits, labels)
  total_loss.append(loss.item())

  print("Attempt at Reconstruction")
  to_rgb(inputs.cpu()[0], logits.cpu().detach().numpy()[0])
  print("Original")
  to_rgb(inputs.cpu()[0], labels.cpu()[0])
print (f"psnr is {sum(total_loss)/len(total_loss)}")

In [ ]:
#Test results
test_transforms = transforms.Compose([])
test_imagefolder = MockData("/content/large_dataset" + '/test' , test_transforms)
test_loader = torch.utils.data.DataLoader(test_imagefolder, batch_size=1, shuffle=False)
best = colorizer_UNETv2().to(device)
best.load_state_dict(torch.load("/content/land_miracle", map_location=torch.device('cpu')))
best.eval()
criterion = PSNRLoss().to(device) # nn.MSELoss().to(device) # Alternatives, PSNR, PA (https://core.ac.uk/download/pdf/151072499.pdf) 
total_loss = []
numb_t = 0
for inputs, labels in test_loader:
  # bw_rgb = greyLAB2RGB(inputs,1).cuda()

  #inputs = inputs
  #labels = labels
  embed = cspdarknet53(inputs)[-1].to(device)

  logits = best(inputs,embed)

  loss = criterion(logits, labels)
  #print(loss.item())
  total_loss.append(loss.item())
  # print("Attempt at Reconstruction")
  # to_rgb(inputs.cpu()[0], logits.cpu().detach().numpy()[0])
  # print("Original")
  # to_rgb(inputs.cpu()[0], labels.cpu()[0])
  numb_t += 1
  if numb_t%10 == 0:
  
    print("Demonstrating Some Test Set Results")
    print("Attempt at Reconstruction")
    to_rgb(inputs.cpu()[0], logits.cpu().detach().numpy()[0])
    print("Original")
    to_rgb(inputs.cpu()[0], labels.cpu()[0])
    print(loss.item())
print (f"psnr is {sum(total_loss)/len(total_loss)}")

## Run on custom image

In [ ]:
  #preprocessing image
  small_size = 128
  rgb = Image.open("/content/lina-loos-04-C1NZk1hE-unsplash.jpg")
  old_size = rgb.size  # old_size is in (width, height) format
  desired_size = max(old_size)
  delta_w = desired_size - old_size[0]
  delta_h = desired_size - old_size[1]
  padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
  new_im = ImageOps.expand(rgb, padding) # padding done
  new_im = new_im.resize((small_size,small_size)) # resizing done
  # https://stackoverflow.com/questions/3228361/using-pil-and-numpy-to-convert-an-image-to-lab-array-modify-the-values-and-then
  npArray = np.asarray(new_im)
  labArray = color.rgb2lab(npArray) # Convert array from RGB into Lab
  np.save("/content/0000000",labArray) # saving

In [ ]:
#image loader
image = np.load("/content/0000000.npy")

image_bw = image[:,:,0]
label = image[:,:,1:]

image_bw = image_bw.reshape(1, 128, 128)
label = np.swapaxes(np.swapaxes(label, 0, 2 ), 2, 1)

#model
model = colorizer_UNETv2().to(device)
model.load_state_dict(torch.load("/content/land_miracle", map_location=torch.device('cpu')))
inputs = torch.from_numpy(image_bw).float()
inputs = torch.unsqueeze(inputs,0)
print(inputs.size())
labels = torch.from_numpy(label).float()

model.eval()
embed = cspdarknet53(inputs)[-1]

logits = model(inputs,embed)
print("Attempt at Reconstruction")
to_rgb(inputs.cpu()[0], logits.cpu().detach().numpy()[0])
print("Original")
to_rgb(inputs.cpu()[0], labels.cpu()[0])